In [ ]:
!pip install datasets transformers[torch] seqeval deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 41.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk("drive/MyDrive/ner_dataset_conll_format_impression_yok")

In [ ]:
#model_names = ["sezinarseven/turkish-medical-field-detection-8","dbmdz/convbert-base-turkish-cased","busecarik/berturk-sunlp-ner-turkish","savasy/bert-base-turkish-ner-cased","dbmdz/convbert-base-turkish-mc4-cased","dbmdz/bert-base-turkish-cased","google-bert/bert-base-multilingual-cased","dbmdz/distilbert-base-turkish-cased","Buseak/penn_berturk_0203_v5","Buseak/model_from_berturk_Feb_5_TrainTestSplit","alierenak/berturk_cased_ner"]
model_names = ["google-t5/t5-large","google-t5/t5-base"]
MODEL_PATH = "/content/drive/MyDrive/ner_model_"

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in model_names]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = cur_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets_list = []
for i in range(len(tokenizers)):
  cur_tokenizer = tokenizers[i]
  a = dataset.map(tokenize_and_align_labels, batched=True)
  tokenized_datasets_list.append(a)


In [ ]:
import datasets
metric = datasets.load_metric("seqeval",trust_remote_code=True)
label_list = dataset["train"].features["ner_tags"].feature.names

<ipython-input-5-169ace8f2e25>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval",trust_remote_code=True)


In [ ]:
import numpy as np
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
    }

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report,f1_score,precision_score,recall_score,accuracy_score
def compute_metrics2(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]

    true_labels_flat = [label for sublist in true_labels for label in sublist]
    pred_labels_flat = [label for sublist in predictions for label in sublist]

    f1 = f1_score(true_labels_flat, pred_labels_flat, average='weighted')
    precision = precision_score(true_labels_flat, pred_labels_flat, average='weighted')
    recall = recall_score(true_labels_flat, pred_labels_flat, average='weighted')
    accuracy = accuracy_score(true_labels_flat, pred_labels_flat)

    return {
   "precision": precision,
   "recall": recall,
   "f1": f1,
  "accuracy": accuracy,
    }



In [ ]:
from transformers import DataCollatorForTokenClassification,Trainer
model_loaded_list = [AutoModelForTokenClassification.from_pretrained(MODEL_PATH + model_name.replace("/","_")+"_impression_yok") for model_name in model_names]
trainer_list = [Trainer(
    model=model_loaded_list[i],
    data_collator=DataCollatorForTokenClassification(tokenizers[i]),
    tokenizer=tokenizers[i],
    compute_metrics=compute_metrics2,
) for i in range(len(model_loaded_list))]

In [ ]:
trainer_list[1].evaluate(tokenized_datasets_list[1]["test"])

{'eval_loss': 1.3291279077529907,
 'eval_precision': 0.8880964309996527,
 'eval_recall': 0.8878481804216909,
 'eval_f1': 0.8872760039385544,
 'eval_accuracy': 0.8878481804216909,
 'eval_runtime': 5.2172,
 'eval_samples_per_second': 47.918,
 'eval_steps_per_second': 6.134}

In [ ]:
prediction_list = [trainer_list[i].predict(tokenized_datasets_list[i]["test"]).predictions for i in range(len(trainer_list))]

In [ ]:
coef_list = [0.46,0.54]
combined_test_preds_proba = prediction_list[0]*coef_list[0] + prediction_list[1]*coef_list[1]
test_labels = tokenized_datasets_list[1]["test"]["labels"]
print(compute_metrics2((combined_test_preds_proba,test_labels)))

{'precision': 0.8913930899320432, 'recall': 0.8909748372099346, 'f1': 0.8903341897480656, 'accuracy': 0.8909748372099346}


In [ ]:
import pickle
with open('hardcoded_dict_tokenized_5.pkl', 'rb') as f:
    hardcoded_dict = pickle.load(f)
print(hardcoded_dict)

{7419: [0.2915601023017903, 0.020460358056265986, 0.33248081841432225, 0.0, 0.3554987212276215], 6570: [0.23655913978494625, 0.0, 0.7634408602150538, 0.0, 0.0], 2067: [0.11371237458193979, 0.0, 0.8862876254180602, 0.0, 0.0], 18: [0.767349260523322, 0.00881683731513083, 0.21302616609783845, 0.004835039817974972, 0.005972696245733789], 7085: [0.15613382899628253, 0.0, 0.7695167286245354, 0.0, 0.07434944237918216], 4082: [0.17079889807162535, 0.0, 0.5454545454545454, 0.0027548209366391185, 0.2809917355371901], 2433: [0.2, 0.0, 0.6, 0.2, 0.0], 20471: [0.058823529411764705, 0.0, 0.9411764705882353, 0.0, 0.0], 2235: [0.13871951219512196, 0.0, 0.5945121951219512, 0.0, 0.26676829268292684], 2462: [0.15738284703801944, 0.01856763925729443, 0.47745358090185674, 0.0, 0.34659593280282935], 8038: [0.14563106796116504, 0.000970873786407767, 0.5174757281553398, 0.0, 0.3359223300970874], 19943: [0.13675213675213677, 0.002849002849002849, 0.8319088319088319, 0.0, 0.02849002849002849], 6915: [0.95051194

In [ ]:
prediction_list[0].shape

(250, 688, 5)

In [ ]:
from scipy.special import softmax
import numpy as np
hardcoded_preds = []
softmax_combined_test_preds_proba = softmax(combined_test_preds_proba,axis=-1)
for i1, text in enumerate(tokenized_datasets_list[0]["test"]["input_ids"]):
    sentence = []
    for i2, id in enumerate(text):
        if id in hardcoded_dict:
            sentence.append(softmax_combined_test_preds_proba[i1][i2]*0 + np.array(hardcoded_dict[id])*1)
        else:
            sentence.append(softmax_combined_test_preds_proba[i1][i2])
    for i in range(688-len(sentence)):
      sentence.append([0.2,0.2,0.2,0.2,0.2])
    hardcoded_preds.append(np.array(sentence))
hardcoded_preds_np = np.array(hardcoded_preds)

In [ ]:
import random
from deap import base, creator, tools, algorithms
# Define the evaluation function
def evaluate(individual):
    x1, x2 = individual
    # Replace this with your actual function
    #coef_list = random_floats_summing_to_one(8).tolist()
    coef_list = [x1,x2]
    combined_test_preds_proba = prediction_list[0]*coef_list[0] + prediction_list[1]*coef_list[1]
    test_labels = tokenized_datasets_list[1]["test"]["labels"]
    result_f1 = compute_metrics2((combined_test_preds_proba,test_labels))["f1"]
    return (result_f1,)

In [ ]:
# Normalize weights to sum to 1
def normalize_weights(individual):
    total = sum(individual)
    return [x / total for x in individual]

In [ ]:
# Create the fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize the toolbox
toolbox = base.Toolbox()

# Attribute generator with normalization
toolbox.register("attr_float", random.uniform, 0, 1)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 lambda: normalize_weights([random.uniform(0, 1) for _ in range(3)]))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the evaluation function
toolbox.register("evaluate", evaluate)

# Register the genetic operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
import multiprocessing
from scipy.optimize import differential_evolution

# Genetic Algorithm parameters
population_size = 200
generations = 20
crossover_probability = 0.7
mutation_probability = 0.2

# Parallelization setup
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

# Create the initial population
population = toolbox.population(n=population_size)

# Apply the genetic algorithm
algorithms.eaSimple(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability,
                    ngen=generations, verbose=True)

# Close the pool
pool.close()
pool.join()

# Extract the best individual after the optimization
best_individual = tools.selBest(population, k=1)[0]
print('Best individual:', best_individual)
print('Best fitness:', best_individual.fitness.values[0])

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local

gen	nevals
0  	200   
1  	146   
2  	143   
3  	154   
4  	158   
5  	164   
6  	157   
7  	152   
8  	148   
9  	168   
10 	153   
11 	136   
12 	166   
13 	163   
14 	159   
15 	136   
16 	158   
17 	156   
18 	140   
19 	168   
20 	147   
Best individual: [0.45924524522676324, 0.40189315664805253, -0.02642922208540941]
Best fitness: 0.8324135927969795


In [ ]:
# Differential Evolution for fine-tuning (optional)
bounds = [(0, 1)] * 3  # Weights should be between 0 and 1

def objective_function(x):
    return -evaluate(normalize_weights(x))[0]  # scipy minimizes, so we negate the function

maxf1 = 0.8549
# Create an initial population for differential evolution
init_list = tools.selBest(population,k=5)
init_pop = [normalize_weights(ind) for ind in init_list]

for i in range(5):
  result = differential_evolution(objective_function, bounds, init=init_pop, maxiter=100000)
  best_weights_de = normalize_weights(result.x)
  print('Iteration', i+1)
  print('Best individual after DE:', best_weights_de)
  print('Best fitness after DE:', -result.fun)

Iteration 1
Best individual after DE: [0.5295403599335491, 0.4627638040143092, 0.007695836052141608]
Best fitness after DE: 0.8324135927969795
Iteration 2
Best individual after DE: [0.5297839243989018, 0.46308213179868585, 0.007133943802412323]
Best fitness after DE: 0.8324135927969795
Iteration 3
Best individual after DE: [0.5297239926831792, 0.4629560445832274, 0.007319962733593366]
Best fitness after DE: 0.8324135927969795
Iteration 4
Best individual after DE: [0.5296187995565257, 0.4628476738442687, 0.007533526599205672]
Best fitness after DE: 0.8324135927969795
Iteration 5
Best individual after DE: [0.5295883638968771, 0.46281195968323785, 0.007599676419884995]
Best fitness after DE: 0.8324135927969795


In [ ]:
evaluate([0.46,0.54])

(0.8903341897480656,)

In [ ]:
for i in range(450,480):
  value = i/1000.0
  print(value," ",1-value," : ",evaluate([value,1-value]))

0.45   0.55  :  (0.890185784914241,)
0.451   0.5489999999999999  :  (0.8902122350217551,)
0.452   0.548  :  (0.890253891657866,)
0.453   0.5469999999999999  :  (0.890266926156881,)
0.454   0.546  :  (0.8902933614905151,)
0.455   0.5449999999999999  :  (0.8902925787630583,)
0.456   0.544  :  (0.8903067786619067,)
0.457   0.5429999999999999  :  (0.890293702396991,)
0.458   0.542  :  (0.8903076933769589,)
0.459   0.5409999999999999  :  (0.8903076689217219,)
0.46   0.54  :  (0.8903341897480656,)
0.461   0.5389999999999999  :  (0.8903070596941025,)
0.462   0.538  :  (0.8903070596941025,)
0.463   0.5369999999999999  :  (0.8902931230916293,)
0.464   0.536  :  (0.8902800466025412,)
0.465   0.5349999999999999  :  (0.8902666863921135,)
0.466   0.534  :  (0.8902539799482374,)
0.467   0.5329999999999999  :  (0.890295610622329,)
0.468   0.532  :  (0.8902965497333921,)
0.469   0.531  :  (0.8902415500781761,)
0.47   0.53  :  (0.8902135983818472,)
0.471   0.529  :  (0.890240609792928,)
0.472   0.528  

In [ ]:
coef_list = [0.529,0.471]
combined_test_preds_proba = prediction_list[0]#*coef_list[0] + prediction_list[1]*coef_list[1]
test_labels = tokenized_datasets_list[0]["test"]["labels"]
print(compute_metrics2((combined_test_preds_proba,test_labels)))


{'precision': 0.8861702641954879, 'recall': 0.8855099849104825, 'f1': 0.8848582084053055, 'accuracy': 0.8855099849104825}


## google-t5/t5-large and google-t5/t5-base models ensemble

coef_list = **[0.288, 0.712]**

{'precision': 0.8128797083839611, 'recall': 0.8872679045092838, **'f1': 0.8484464172479391**, 'accuracy': 0.8938628987312723}

## impression_yok modelleri ensemble

coef_list = [0.529,0.471]

{'precision': 0.7962243198223209, 'recall': 0.8717325227963526, 'f1': 0.8322692977365062, 'accuracy': 0.8906757656910591}


### seqeval yok

coef_list = [0.46,0.54]

{'precision': 0.8913930899320432, 'recall': 0.8909748372099346, 'f1': 0.8903341897480656, 'accuracy': 0.8909748372099346}